In [8]:
import os
import pickle
import numpy as np
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Veriyi Yükleme ve Ön işleme Fonksiyonları
def load_data(file_path):
    df = pd.read_csv(file_path)
    # Sadece gerekli kolonları seçerek basitleştirme
    selected_columns = ["TransactionAmount", "CustomerAge", "TransactionDuration", "LoginAttempts", "AccountBalance"]
    df = df[selected_columns]
    # Hedef sütunu oluştur: Hedef olarak TransactionAmount'ın belirli bir eşikten büyük olup olmadığını kullanacağız
    df['HighTransaction'] = (df['TransactionAmount'] > 1000).astype(int)
    return df

def preprocess_data(df):
    # Gerekli ön işleme adımları (eksik verilerin temizlenmesi, dönüşümler vs)
    df.dropna(inplace=True)  # Eksik verileri kaldır
    return df

def split_data(df):
    X = df.drop("HighTransaction", axis=1)
    y = df["HighTransaction"]
    return train_test_split(X, y, test_size=0.3, random_state=42)  # Test setini %30 olarak ayır

def split_and_save_data(file_path):
    # Veriyi yükle
    df = pd.read_csv(file_path)
    
    # Veriyi %70 eğitim, %30 test olacak şekilde ayır
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
    
    # Eğitim ve test verilerini aynı dizine kaydet
    train_path = file_path.replace('.csv', '_train.csv')
    test_path = file_path.replace('.csv', '_test.csv')
    
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    
    print(f"Eğitim verisi kaydedildi: {train_path}")
    print(f"Test verisi kaydedildi: {test_path}")

# Model Eğitim ve Değerlendirme Fonksiyonları
def train_model(X_train, y_train, balance_classes=True, class_sampling_factors=None):
    # H2O sunucusunu başlat
    h2o.init()

    # Eğitim verilerini H2OFrame'e dönüştür
    train_data = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
    train_data[y_train.name] = train_data[y_train.name].asfactor()  # Hedef sütunu kategorik yap

    # AutoML modelini tanımla ve eğit
    aml = H2OAutoML(max_runtime_secs=180, seed=42, stopping_metric='logloss', stopping_rounds=5, max_models=5, balance_classes=balance_classes, class_sampling_factors=class_sampling_factors)
    aml.train(y=y_train.name, training_frame=train_data)

    return aml

def save_model(model, model_path):
    model_path = h2o.save_model(model=model.leader, path=model_path, force=True)
    print(f"Model saved to {model_path}")

def evaluate_model(model, X_test, y_test):
    # Test verilerini H2OFrame'e dönüştür
    test_data = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))
    test_data[y_test.name] = test_data[y_test.name].asfactor()

    # Modeli test verileri üzerinde değerlendirin
    performance = model.leader.model_performance(test_data)
    print(performance)

    # Test verisi üzerindeki tahminler
    predictions = model.leader.predict(test_data).as_data_frame()
    print(predictions)

    # Tahminlerin değerlendirilmesi
    y_pred = predictions['predict'].astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    print("Accuracy Score:", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred))

    return accuracy, report

def dynamic_class_weight_training(X_train, y_train, X_test, y_test, results_path="results.txt", models_dir="models"):
    # Model dosyalarının kaydedileceği klasörü oluştur
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    best_model = None
    best_f1_score = 0
    acceptable_f1_range = (0.75, 0.85)  # Uygun gördüğümüz f1-score aralığı
    class_weights = np.linspace(0.5, 5.0, 10)  # 0.5 ile 5.0 arasında 10 farklı class weight faktörü denenecek
    attempts = 0  # Eğitim deneme sayısı

    for weight in class_weights:
        print(f"Class weight faktörü: {weight}")
        model = train_model(X_train, y_train, balance_classes=True, class_sampling_factors=[1.0, weight])
        accuracy, report = evaluate_model(model, X_test, y_test)

        f1_score_1 = report['1']['f1-score']  # Sınıf 1 için f1-score
        print(f"Sınıf 1 için F1-Score: {f1_score_1}")

        # Save the model in pickle format
        model_path = os.path.join(models_dir, f"model_weight_{weight:.2f}.pkl")
        save_model(model, model_path)
        print(f"Model kaydedildi: {model_path}")

        # Sonuçları dosyaya yaz
        with open(results_path, "a") as file:
            file.write(f"Class weight: {weight} # Accuracy: {accuracy} # F1-Score (Class 1): {f1_score_1} # Model Path: {model_path}\n")

        # Eğer f1-score uygun aralıkta ise dur
        if acceptable_f1_range[0] <= f1_score_1 <= acceptable_f1_range[1]:
            print(f"Uygun F1-Score elde edildi: {f1_score_1}, Class weight faktörü: {weight}")
            best_model = model
            break

        # Daha iyi bir f1-score elde edilirse en iyi modeli güncelle
        if f1_score_1 > best_f1_score:
            best_f1_score = f1_score_1
            best_model = model

        attempts += 1
        # Eğer 5 denemeden sonra istenilen sonuca ulaşılamadıysa, döngüğyü durdur ve farklı yaklaşımlar dene
        if attempts >= 5:
            print("5 deneme yapıldı, uygun sonuç elde edilemedi. Farklı class weight faktörleri deneniyor.")
            class_weights = np.linspace(5.5, 10.0, 5)  # Yeni bir class weight aralığı denenecek
            attempts = 0  # Deneme sayısını sıfırla

    return best_model

# Ana fonksiyon
def main():
    # Veri Yükleme
    data_path = "data/bank_transactions_data_2.csv"
    
    # Veriyi Ayırma ve Kayıt Etme
    split_and_save_data(data_path)

    # Eğitim verisini yükle
    train_data_path = "data/bank_transactions_data_2_train.csv"
    df = load_data(train_data_path)

    # Veri Ön İşleme
    df = preprocess_data(df)

    # Veri Ayırma
    X_train, X_test, y_train, y_test = split_data(df)

    # Dinamik Class Weight ile Model Eğitimi
    best_model = dynamic_class_weight_training(X_train, y_train, X_test, y_test)
    print("En iyi model eğitildi ve seçildi.")

if __name__ == "__main__":
    main()


Eğitim verisi kaydedildi: data/bank_transactions_data_2_train.csv
Test verisi kaydedildi: data/bank_transactions_data_2_test.csv
Class weight faktörü: 0.5
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,21 mins 30 secs
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_deniz_jlepsa
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.770 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:14:29.478: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.0024982307416568554
RMSE: 0.049982304285185325
LogLoss: 0.01250884843656639
Mean Per-Class Error: 0.02
AUC: 0.9992445328031809
AUCPR: 0.9892000361257741
Gini: 0.9984890656063619

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6948905109489051
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      503  0    0        (0.0/503.0)
1      1    24   0.04     (1.0/25.0)
Total  504  24   0.0019   (1.0/528.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  ------

d:\users\deniz\appdata\local\programs\python\python39\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


     predict        p0        p1
0          0  1.000000  0.000000
1          1  0.096203  0.903797
2          0  1.000000  0.000000
3          0  1.000000  0.000000
4          0  1.000000  0.000000
..       ...       ...       ...
523        0  1.000000  0.000000
524        0  1.000000  0.000000
525        0  1.000000  0.000000
526        0  1.000000  0.000000
527        0  1.000000  0.000000

[528 rows x 3 columns]
Accuracy Score: 0.9924242424242424
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       503
           1       1.00      0.84      0.91        25

    accuracy                           0.99       528
   macro avg       1.00      0.92      0.95       528
weighted avg       0.99      0.99      0.99       528

Sınıf 1 için F1-Score: 0.9130434782608695
Model saved to D:\Users\deniz\Downloads\eurotech-main(1)\eurotech-main\5.Hafta\models\model_weight_0.50.pkl\DRF_1_AutoML_1_20241110_201429
Model kaydedi

...

Sınıf 1 için F1-Score: 0.0
Model saved to D:\Users\deniz\Downloads\eurotech-main(1)\eurotech-main\5.Hafta\models\model_weight_4.50.pkl\GBM_2_AutoML_9_20241110_202721
Model kaydedildi: models\model_weight_4.50.pkl
Class weight faktörü: 5.0
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,36 mins 12 secs
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_deniz_jlepsa
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.740 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:29:11.646: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.001893921228908151
RMSE: 0.043519205288104136
LogLoss: 0.023098861701549067
AUC: 0.9998409542743539
AUCPR: 0.9969807737613723
Gini: 0.9996819085487079
Null degrees of freedom: 527
Residual degrees of freedom: 525
Null deviance: 207.6201491265345
Residual deviance: 24.392397956832127
AIC: 30.392397956832127

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9998349253792045
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      503  0    0        (0.0/503.0)
1      1    24   0.04     (1.0/25.0)
Total  504  24   0.0019   (1.0/528.0)

Maximum Me

d:\users\deniz\appdata\local\programs\python\python39\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


     predict        p0        p1
0          0  0.999996  0.000004
1          1  0.000154  0.999846
2          0  0.999996  0.000004
3          0  0.999996  0.000004
4          0  0.999996  0.000004
..       ...       ...       ...
523        0  0.999996  0.000004
524        0  0.999996  0.000004
525        0  0.999996  0.000004
526        0  0.999996  0.000004
527        0  0.999996  0.000004

[528 rows x 3 columns]
Accuracy Score: 0.9981060606060606
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       503
           1       1.00      0.96      0.98        25

    accuracy                           1.00       528
   macro avg       1.00      0.98      0.99       528
weighted avg       1.00      1.00      1.00       528

Sınıf 1 için F1-Score: 0.9795918367346939
Model saved to D:\Users\deniz\Downloads\eurotech-main(1)\eurotech-main\5.Hafta\models\model_weight_5.00.pkl\StackedEnsemble_BestOfFamily_1_AutoML_10_202